In [ ]:
from gurobipy import *
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from gurobipy import *
import pandas as pd

In [ ]:
m1 = Model('VRPTW')

In [ ]:


# m1 = Model('vrptw')

# # 20 customer -> 1.1 minute
# # 30 customers -> 5 minute     1.69 minutes
# # 40                           2.45 min
# # 50 customer -> 160 minute


def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw





In [ ]:
customers = 100
file = "RC106.csv"
df = pd.read_csv(file)
start = 0
speed = 1 # km/min
df['XCOORD.'] = df['XCOORD.'] - df['XCOORD.'][0]
df['YCOORD.'] = df['YCOORD.'] - df['YCOORD.'][0]
# df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
# df.reset_index(inplace = True)

serviceTime = df['SERVICE TIME'][1]
#remove unreachable if(starting time + travelling > end )


print(len(df))
# df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
timeWindows = [] 
for i in range(0,customers):
    st = df['READY TIME'][i+1]
    if(st%10 != 0):
        st=st + (10 - df['READY TIME'][i+1]%10)
    et = df['DUE DATE'][i+1]
    if(et%10 != 0):
        et = et +(10 - df['DUE DATE'][i+1]%10)
    tw = [st,et]
    timeWindows.append(tw)

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i,10))

time_from_source = []
for i in range(0,customers):
    time_from_source.append(round(math.sqrt((df['XCOORD.'][i+1])**2  + (df['YCOORD.'][i+1])**2 ),2))

timeMatrix = []

for i in range(0,customers):
    timeMatrix.append([0]*customers)
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
#             c= int((abs(df['XCOORD.'][i+1]-df['XCOORD.'][j+1]))+abs(df['YCOORD.'][i+1]-df['YCOORD.'][j+1]))
            euclidean_distance= math.sqrt(  (df['XCOORD.'][i+1]-df['XCOORD.'][j+1])**2  + (df['YCOORD.'][i+1]-df['YCOORD.'][j+1])**2 )
            timeMatrix[i][j] = round(euclidean_distance,2)
            timeMatrix[j][i] = round(euclidean_distance,2)
  
                
# print("timeWindows",timeWindows)
# print("timeMatrix",timeMatrix)

In [ ]:
# serviceTime = 0

# customers = 5
# timeWindows = []
# for i in range(0,customers):
#     st = random.randint(1,2)*10
#     et = random.randint(1,3)*10 + st
#     tw = [st,et]
#     timeWindows.append(tw)

# print("timeWindows =", timeWindows)
# print()

# discreteTimeWindows = []
# for i in timeWindows:
#     discreteTimeWindows.append(discreteTime(i, 10))

# # print("discreteTimeWindows ", discreteTimeWindows)
# # print()



# timeMatrix = []
# for i in range(0,customers):
#     timeMatrix.append([0]*customers)
# for i in range(0,customers):
#     for j in range(0,customers):
#         if(i==j):
#             timeMatrix[i][j] = 0
#         elif (i<j):
#             timeMatrix[i][j] = random.randint(1,10)*10
#             timeMatrix[j][i] = timeMatrix[i][j]





In [ ]:
count = 0
nodeCount = []  # assigns a number to node using count and stores its value

for i in range(0,len(discreteTimeWindows)):
    c = []
    for j in range(0, len(discreteTimeWindows[i])):
        c.append(count+1)
        count += 1
    nodeCount.append(c)

# print("Node Count ", nodeCount)
# print()


edgeFlows = {}
edgeNumber = 0
x = [] 
edges = {}
edgesList = []  # stores edge number going out of ith node
edgesList.append([])  # for s
# solver = pywraplp.Solver.CreateSolver('GLOP')

In [ ]:
e = []
for i in range(0, len(nodeCount)):
            reaching_time = time_from_source[i]
            if(reaching_time <= discreteTimeWindows[i][0]):
                x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
                edges['x[%i]' % edgeNumber] = (0, nodeCount[i][0])
                edgeFlows[edgeNumber] = (0, nodeCount[i][0])
                e.append(edgeNumber)
                edgeNumber += 1
               
    
            else:
                for j in range(0, len(discreteTimeWindows[i])):
                        if(reaching_time <= discreteTimeWindows[i][j]):
                            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
                            edges['x[%i]' % edgeNumber] = (0, nodeCount[i][j])
                            edgeFlows[edgeNumber] = (0, nodeCount[i][j])
                            e.append(edgeNumber)
                            edgeNumber += 1
                            break
edgesList[0] = e

for i in range(0, len(nodeCount)):
    for j in range(0, len(nodeCount[i])-1):
        e = []
#         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
#         m1.update()
        edges['x[%i]' % edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        edgeFlows[edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        # add edge number in edge list
        e.append(edgeNumber)
        edgesList.append(e)
        edgeNumber += 1
    edgesList.append([])


    
for i in range(len(discreteTimeWindows)):
    tu = nodeCount[i][-1]
    start_utime = discreteTimeWindows[i][0]
    end_utime = discreteTimeWindows[i][-1]

    # stores edges reachable from u
    for k in range(0, len(discreteTimeWindows)):
        sv = nodeCount[k][0]

        start_vtime = discreteTimeWindows[k][0]
        end_vtime = discreteTimeWindows[k][-1]


        if(i != k and start_utime + timeMatrix[i][k]+serviceTime > end_vtime):
            print("",end="")
#             print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)

              # not reachable

        elif(i != k and start_vtime > end_utime + timeMatrix[i][k]+ serviceTime):
            # edge bw (u,tu) -> (v,sv)
#             print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
            
            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
            edgeFlows[edgeNumber] = (tu, sv)
#             print(tu, "->", sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif(i != k and start_vtime == end_utime + timeMatrix[i][k]+serviceTime):
#             print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
            

            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
#             print(tu, "->", sv)
            edgeFlows[edgeNumber] = (tu, sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1

            
        elif (i != k and start_vtime  < end_utime + timeMatrix[i][k]+serviceTime):

                firstReachableNode_index = 0  # first such that node >= sv
                for j in range(0, len(discreteTimeWindows[i])):
                    if(timeMatrix[i][k] + discreteTimeWindows[i][j]+serviceTime >= start_vtime):
                        firstReachableNode_index = j
                        break
                if(firstReachableNode_index > 0):
                    firstReachableNode_index=firstReachableNode_index-1
                # now starting from firstReachableNode make edges until feasible
                for l in range(firstReachableNode_index, len(discreteTimeWindows[i])):
                    e = []
                    reachingTime = discreteTimeWindows[i][l]+timeMatrix[i][k]+serviceTime
                    for r in range(0, len(discreteTimeWindows[k])):
                        if(discreteTimeWindows[k][r] >= reachingTime):
                            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
                            edges['x[%i]' % edgeNumber] =(nodeCount[i][l], nodeCount[k][r])  # u -> v reachable
                            edgeFlows[edgeNumber] = (nodeCount[i][l], nodeCount[k][r])
                            e.append(edgeNumber)
                            edgesList[nodeCount[i][l]].extend(e)
                            edgeNumber += 1
                            break




#         elif (i != k and start_vtime < end_utime + timeMatrix[i][k]+serviceTime):
           
#             if(start_utime > start_vtime-timeMatrix[i][k]-serviceTime):
#                 count_u = 0  # first such that node >= sv
#                 count_v = 0
                

#                 for r in range(0, len(discreteTimeWindows[k])):
#                     if(timeMatrix[i][k]+serviceTime+discreteTimeWindows[i][0]<=discreteTimeWindows[k][r]):
#                             count_v = r
#                             break

#                 while(count_u < len(discreteTimeWindows[i]) and count_v < len(discreteTimeWindows[k])):
#                     x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
#                     edges['x[%i]' % edgeNumber] =(nodeCount[i][count_u],nodeCount[k][count_v])  # u -> v reachable
#                     edgeFlows[edgeNumber] = (nodeCount[i][count_u],nodeCount[k][count_v])
#                     e.append(edgeNumber)
#                     edgesList[nodeCount[i][count_u]].extend(e)
#                     edgeNumber += 1
#                     count_u+=1
#                     count_v+=1
                    
#             else:
                
#                 count_u = 0  # first such that node >= sv
#                 count_v = 0
#                 for j in range(0, len(discreteTimeWindows[i])):
#                     if(timeMatrix[i][k]+discreteTimeWindows[i][j]+serviceTime == start_vtime):
#                         count_u = j  
#                         break
#                     elif(timeMatrix[i][k] + discreteTimeWindows[i][j]+serviceTime > start_vtime):
#                         count_u = j-1  
#                         break
                
#                 while(count_u < len(discreteTimeWindows[i]) and count_v < len(discreteTimeWindows[k])):

#                     x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
#                     edges['x[%i]' % edgeNumber] =(nodeCount[i][count_u], nodeCount[k][count_v])  # u -> v reachable
#                     edgeFlows[edgeNumber] = (nodeCount[i][count_u],nodeCount[k][count_v])
#                     e.append(edgeNumber)
#                     edgesList[nodeCount[i][count_u]].extend(e)
#                     edgeNumber += 1


#                     count_u+=1
#                     count_v+=1
            
            
            
            




# print('edge list b4 s t', edgesList)
# print()

# into t (except s)
e = []
for i in range(0, len(nodeCount)):
    x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
    edges['x[%i]' % edgeNumber] = (nodeCount[i][-1], -1)
    edgeFlows[edgeNumber] = (nodeCount[i][-1],-1)
    # add edge number in edge list
    edgesList[nodeCount[i][-1]].append(edgeNumber)
    edgeNumber += 1
edgesList.append([])


noc = 0
# contraint for outgoing flow for a customer
for i in range(0, len(nodeCount)):  # choose one list from nodeCount
    #     contraint = [0] * (edgeNumber)
        coeff = [] # store where coeff = 1
        for j in range(0, len(nodeCount[i])):  # se
            node = nodeCount[i][j]
            for k in range(0, len(edgesList[node])):
                e = edgesList[node][k]
                u = edges['x[%i]' % e][0]
                v = edges['x[%i]' % e][1]
                    # print("u", u, "v", v)
                if(v not in nodeCount[i]):
                        coeff.append(x[e])

        m1.addConstr(( sum(coeff) == 1  ))
        noc+=1

#     print("outgoing edge lisst :")

incomingEdgeList = []
for i in range(0, len(edgesList)):
        incomingEdgeList.append([])  # adding lists in a list
for i in edges:
        v = edges[i][1]  # u -> v

        edge_no = int(i[2:-1])
        incomingEdgeList[v].append(edge_no)

#     print("incoming edge list: ", incomingEdgeList)
#     print()

    #  constraint for incoming for a customer
for i in range(0, len(nodeCount)):
        coeff = []
        contraint = [0] * (edgeNumber)  # indicates number of edges
        for j in range(0, len(nodeCount[i])):
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                u = edges['x[%i]' % e][0]
                v = edges['x[%i]' % e][1]
                if(u not in nodeCount[i]):
                        coeff.append(x[e])
        m1.addConstr(( sum(coeff) == 1  ))
        noc+=1

# constraint for a node incoming = outgoing
for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])):
            coeff = []
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                coeff.append(x[e])
            for k in range(0, len(edgesList[node])):
                e = edgesList[node][k]
                coeff.append(-x[e])
            m1.addConstr(( sum(coeff) == 0 ))
            noc+=1  

In [ ]:
objectivefun = []
for i in range(0, len(edgesList[0])):
    objectivefun.append(x[edgesList[0][i]])
obj = sum(objectivefun)
m1.setObjective(obj,GRB.MINIMIZE)
                

    
m1.optimize()
print("Runtime is :",m1.Runtime)
print("objective value : ",m1.objVal)

In [ ]:
print(file)
print("Vehicles : ",m1.objVal)
print("Runtime is :",m1.Runtime," seconds")

In [ ]:
# for v in m1.getVars():
#     if(v.x == 1):
#         print(v.varName)
# #     print('%s: %g' % (v.varName,v.x))

## 